In [1]:
import pandas as pd
import numpy as np
import math
from math import cos, sin, pi

df_T = pd.read_csv('data/T_amb.csv', header=None)       # 辐射体温度/环境温度
df_e = pd.read_csv('data/cooler.csv', header=None)        # 辐射体发射率
df_a = pd.read_csv('data/cooler.csv', header=None)        # 辐射体吸收率
df_cc = pd.read_csv('data/cc.csv', header=None)        # 云层覆盖率
df_trans = pd.read_csv('data/trans.csv', header=None)  

################################ 常量定义 ################################
DELTA_THETA = 1 # 角度间隔
DELTA_TIME = 60.0 # 时间间隔
T_C = 0.9   # PE膜透过率
CC = 0.15  # 云层覆盖率
E_CLOUD = 0.75 #云层发射率
LAMBDA = df_e.iloc[:-1,0].values # 辐射体发射率对应的波长
D_LAMBDA = df_e.iloc[1:,0].values - LAMBDA # 前后项波长差
D_THETA = np.arange(0,90,DELTA_THETA) # 角度
T_ATM = np.full(LAMBDA.shape, 300 ,dtype=np.float64) # 大气温度
THETA = 5/180*np.pi # 太阳高度角
CC = df_cc.iloc[:,1].values/100 # 云层覆盖率

In [2]:

def n_I_BB(W:float,T:float)->float:
    '''
    计算黑体光谱辐射力
    W:辐射体波长
    T:辐射体温度
    '''
    C_1 = 3.743e8
    C_2 = 1.439e4
    return (C_1 * W ** (-5) ) / (np.exp(C_2 / (W * T)) -1)

def P_rad(T1:float = 300)->float:
    '''
    计算辐射功率
    T1:辐射体温度
    调用n_I_BB函数通过辐射体波长和温度计算黑体光谱辐射力，再乘以辐射体发射率、波长差与透过率，最后求和
    输出单个时间点的全波段辐射功率
    '''
    np_df_e = np.array(df_e)
    W = np.array(df_e.iloc[:-1,0]).reshape(1319,)      #辐射体发射率对应的波长
    T = np.full((1319,),T1)       # 辐射体温度
    t_c = np.full((1319,),T_C)     # PE膜透过率0.9
    delta_W = np_df_e[1:,0] - np_df_e[:-1,0]    # 前后项波长差
    np_cooler = np.array(df_e.iloc[:-1,1])        # 辐射体发射率
    rad = np.apply_along_axis(n_I_BB,0,W,T) * np_cooler * delta_W * t_c # 计算辐射力*辐射体发射率*波长差*PE膜透过率 需要保持维度一致
    return rad.sum()

def E_rad(delta_t:int = 60, is_sum:bool = True, is_all = False) -> list :
    '''
    is_sum = True 返回总辐射功率
    is_sum = False 返回每个时间点的辐射功率
    delta_t = 60 为默认时间间隔(s),当delta_t = 1时,返回瞬时辐射功率
    '''
    P_list = []
    np_df_T = np.array(df_T)
    if is_all:
        for i in range(len(np_df_T[:,1])):
            P_list.append(P_rad(np_df_T[i,1]))
    else: 
        for i in range(len(np_df_T[:,1])):
            if np_df_T[i,1] < np_df_T[i,2]:            # 比较辐射体温度与环境温度大小
                P_list.append(P_rad(np_df_T[i,1]))        # 若辐射体温度＜环境温度，则计算制冷功率
            else:
                P_list.append(0)                          # 若辐射体温度＞环境温度，则制冷功率为0

    # 进行向量化计算
    P_list = np.array(P_list)
    delta_t = np.full((len(P_list),), delta_t)
    P_list = P_list * delta_t
    
    if is_sum:
        return sum(P_list)
    else:
        return P_list
    

def P_atm(theta = (5/180*np.pi),t_atm=300,is_test = False,is_sum = True):
    alpha = np.array(df_a.iloc[:-1,1])
    e_atm = 1-(np.array(df_trans.iloc[:-1,1]))**(1/np.cos(theta))
    cc = np.full(alpha.shape,CC[0])
    e_cloud = np.full(alpha.shape,E_CLOUD)
    e_lambda = np.apply_along_axis(n_I_BB,0,LAMBDA,t_atm)
    cos_sin = np.full(alpha.shape,np.cos(theta)*np.sin(theta))
    args = e_atm*(1-cc)+cc*e_cloud
    if is_test:
        args = 1
    if is_sum:
        res = []
        for j in D_THETA:
            res.append(P_atm(j/180*np.pi,t_atm,is_test,is_sum=False)*DELTA_THETA/180*np.pi)
        result = np.array(res).sum()*2*T_C
        return result
    return (alpha*(args)*e_lambda*D_LAMBDA*cos_sin).sum()


def E_atm(is_test = False):
    res = []
    num = 2
    if is_test:
        num = 1
    for i in df_T.iloc[:,num]:
        res.append(P_atm(t_atm = i,is_test=is_test)*60)
    return sum(res)

In [4]:
P_rad(),P_atm(),E_rad(),E_atm()

(264.3381504291769, 204.99792704189915, 13563666.537712649, 11136288.17949053)